# Neural Networks using NumPy
*Jacob Nelson and Yen Lee Loh, 2021-5-31; 2022-12-30*

This Jupyter Lab notebook demonstrates how one may construct simple neural networks using pure NumPy.

---
## 1. Single-Layer Perceptron

Consider a single-layer perceptron described by the feedforward equations
$u_1 = x_0 w_{01}, x_1 = \tanh u_1, F=(x_1-y)^2$:

    x0  ----------------> u1 ----------------> x1 ----------------> F
    w01 ---------------/                       y  --------------/

Here x0 is the input, w01 are the weights, u1 is a hidden node, x1 is the model output, y is the training output, and F is the loss.  We may calculate the gradient of F with respect to every node using backpropagation:

    dF/dx0  <-------- dF/du1 <------------- dF/dx1 <-------------- dF/dF=1
    dF/dw01 <--------/                      dF/dy  <--------------/

This allows us to calculate the gradient of $F$ with respect to the learnable parameters (weights), which allow us to train the network by adjusting the weights.

In [4]:
import numpy as np
np.set_printoptions (precision=4)
rng = np.random.default_rng(12345)
def mystr(a): return np.array2string(a.flatten().round(1), formatter={'float_kind': '{:+.1f}'.format})
def sech2(x): return np.cosh(x)**(-2)

xnd = np.array( [[0,0,1],[0,1,1],[1,0,1],[1,1,1]] ) # training inputs (4x3)
ynd = np.array( [[-1],   [1],    [1],    [1]    ] ) # training outputs (4x1)
N,D0 = xnd.shape
N,D1 = ynd.shape

w01 = rng.normal(size=(D0,D1))  # random weights
tau = 0.01                      # learning rate
print ('{:10}{:20}{:30}{:10}'.format ('', 'Initial weights W', 'Training outputs Y', ''))
print ('{:10}{:20}{:30}{:10}'.format ('', mystr(w01), mystr(ynd), ''))
print ()
print ('{:10}{:20}{:30}{:10}'.format ('Epoch', 'Weights W', 'Model outputs x1', 'Loss F'))
#======== Loop over training epochs
for t in range(10000):
  #======== Load a batch of training data (in this case we just work with the entire dataset)
  x0 = xnd                    # N*D0
  y = ynd                     # N*D1
  #======== Feedforward
  u1 = x0 @ w01               # N*D1
  x1 = np.tanh (u1)           # N*D1
  F = (x1-y).T @ (x1-y)       # scalar
  #======== Backpropagate
  dFdx1  = 2*(x1-y)           # N*D1
  dFdu1  = sech2(u1) * dFdx1  # N*D1
  dFdx0  = w01 @ dFdu1.T      # N*D0 (actually we didn't need to compute this)
  dFdw01 = x0.T @ dFdu1       # D0*D1, obtained from (D0*N) dotted with (N*D1)
  #======== Train
  w01 -= tau * dFdw01
  if t%1000==0:
    print ('{:<10d}{:20s}{:30s}{:.4f}'.format( t, str(w01.flatten().round(1)), mystr(x1), F[0,0] ))
    

          Initial weights W   Training outputs Y                      
          [-1.4 +1.3 -0.9]    [-1  1  1  1]                           

Epoch     Weights W           Model outputs x1              Loss F    
0         [-1.4  1.3 -0.8]    [-0.7 +0.4 -1.0 -0.8]         7.5486
1000      [ 2.4  2.5 -1.1]    [-0.8 +0.9 +0.9 +1.0]         0.0695
2000      [ 2.9  2.9 -1.3]    [-0.9 +0.9 +0.9 +1.0]         0.0335
3000      [ 3.1  3.1 -1.4]    [-0.9 +0.9 +0.9 +1.0]         0.0218
4000      [ 3.2  3.2 -1.5]    [-0.9 +0.9 +0.9 +1.0]         0.0161
5000      [ 3.4  3.4 -1.6]    [-0.9 +0.9 +0.9 +1.0]         0.0127
6000      [ 3.5  3.5 -1.6]    [-0.9 +1.0 +1.0 +1.0]         0.0105
7000      [ 3.5  3.5 -1.7]    [-0.9 +1.0 +1.0 +1.0]         0.0089
8000      [ 3.6  3.6 -1.7]    [-0.9 +1.0 +1.0 +1.0]         0.0078
9000      [ 3.7  3.7 -1.7]    [-0.9 +1.0 +1.0 +1.0]         0.0069


---
## Example 1: Single-Layer Perceptron

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

def printTable(a):           # a is a numpy.array
    [imax,jmax] = np.shape(a);
    for i in range(imax):
        print ("\t", end='')
        for j in range(jmax): 
            print ("{:.3f}".format(a[i,j]), end='\t')
        print ()

def g(x): return 1/(1+np.exp(-x)) # g is a sigmoid function
def h(g): return g*(1-g)          # h is such that g'(x) = h(g(x))

X = np.array( [[0,0,1],[0,1,1],[1,0,1],[1,1,1]] )  # define training inputs
Y = np.array( [[0,1,1,1]] ).T                      # define training outputs (column vector)
nmax,dmax = X.shape

print ("\nTraining neural network ...\n")
np.random.seed(1)
w0 = 2*np.random.random((3,1)) - 1   # init weights randomly
for j in range(30000):
    x0 = X               # entire set of training data
    x1 = g( x0.dot(w0) ) # feedforward to level 1 neurons (entire dataset at once)
    e1 = Y - x1          # level 1 error
    d1 = e1 * h(x1)      # level 1 delta
    w0 += x0.T.dot(d1)   # update layer0-layer1 couplings
    if (j%10000) == 0:
        print ("Iteration {:8d}\tError e1 = {:6.4f}".format (j, np.mean(np.abs(e1))  )) 

print ("\nWeights:\n")
print ("\tw1\tw2\tw3")
print ("\t-------\t-------\t-------\t-------\t-------\t")
printTable (w0.T)
        
print ("\nTraining dataset and prediction accuracy:\n")
print ("\tx1\tx2\tx3\tYPred\tyTrain")
print ("\t-------\t-------\t-------\t-------\t-------\t")
printTable (np.hstack ((X, x1, Y)) )


Training neural network ...

Iteration        0	Error e1 = 0.5853
Iteration    10000	Error e1 = 0.0092
Iteration    20000	Error e1 = 0.0065

Weights:

	w1	w2	w3
	-------	-------	-------	-------	-------	
	9.805	9.805	-4.670	

Training dataset and prediction accuracy:

	x1	x2	x3	YPred	yTrain
	-------	-------	-------	-------	-------	
	0.000	0.000	1.000	0.009	0.000	
	0.000	1.000	1.000	0.994	1.000	
	1.000	0.000	1.000	0.994	1.000	
	1.000	1.000	1.000	1.000	1.000	


---

## Example 2: Two-Layer Perceptron

In [2]:
import numpy as np

X = np.array([[0,0,1],[0,1,1],[1,0,1],[1,1,1]])
Y = np.array([[0],[1],[1],[0]])

np.random.seed(1)
w0 = 2*np.random.random((3,4)) - 1   # weights from layer 0 to layer 1 (matrix)
w1 = 2*np.random.random((4,1)) - 1   # weights from layer 1 to layer 2 (matrix)

for j in range(60000):
    x0 = X
    x1 = g( x0.dot(w0) )  # Feedforward through layer 1
    x2 = g( x1.dot(w1) )  # Feedforward to layer 2
    e2 = Y - x2           # Error at level 2
    d2 = e2 * h(x2)       # Check confidence, mute confidence answers
    
    e1 = d2.dot (w1.T)    # Backpropagate: how much did each x1 value contribute to e2?    
    d1 = e1 * h(x1)       # Check confidence of x1
    
    w1 += x1.T.dot(d2)    # Update weights
    w0 += x0.T.dot(d1)
   
    if (j%10000) == 0:
        print ("Iteration {:8d}\tError e1 = {:6.4f}".format (j, np.mean(np.abs(e1))  )) 

print ("\nWeights w0 (between input and hidden layer):\n")
printTable (w0)
print ("\nWeights w1 (between hidden layer and output):\n")
printTable (w1)
print ("\nTraining dataset and prediction accuracy:\n")
print ("\tx1\tx2\tx3\tz1\tz2\tz3\tz4\tYPred\tyTrain")
print ("\t-------\t-------\t-------\t-------\t-------\t-------\t-------\t-------\t--------")
printTable (np.hstack ((X, x1, x2, Y)) )

Iteration        0	Error e1 = 0.0813
Iteration    10000	Error e1 = 0.0005
Iteration    20000	Error e1 = 0.0003
Iteration    30000	Error e1 = 0.0002
Iteration    40000	Error e1 = 0.0001
Iteration    50000	Error e1 = 0.0001

Weights w0 (between input and hidden layer):

	4.601	4.172	-6.310	-4.197	
	-2.584	-5.814	-6.608	-3.684	
	0.975	-2.027	2.529	5.844	

Weights w1 (between hidden layer and output):

	-6.968	
	7.141	
	-10.319	
	7.861	

Training dataset and prediction accuracy:

	x1	x2	x3	z1	z2	z3	z4	YPred	yTrain
	-------	-------	-------	-------	-------	-------	-------	-------	--------
	0.000	0.000	1.000	0.726	0.116	0.926	0.997	0.003	0.000	
	0.000	1.000	1.000	0.167	0.000	0.017	0.897	0.997	1.000	
	1.000	0.000	1.000	0.996	0.895	0.022	0.838	0.997	1.000	
	1.000	1.000	1.000	0.952	0.025	0.000	0.115	0.004	0.000	
